In [1]:
#Basic libraries
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pd.read_csv('Theseus_data_19.5.2021_FINAL.csv', sep = '|')
df.sort_values(['year'], inplace = True)
print('Data length:', len(df))

#Calculate the length as the total words of English and Finnish abstracts
df['len'] = df['en'].str.split().str.len() + df['fi'].str.split().str.len() 

Data length: 176738


In [3]:
#Clean data
#original_language
df.loc[df['original_language'].str.contains('fi', na = False), 'original_language'] = 'fin'
df.loc[df['original_language'].str.contains('en', na = False), 'original_language'] = 'eng'
df.loc[df['original_language'].str.contains('sv', na = False), 'original_language'] = 'swe'
df.loc[df['original_language'].isnull(), 'original_language'] = np.NaN
df.loc[~(df['original_language'].str.contains('fin', na = False) |
         df['original_language'].str.contains('eng', na = False) |
         df['original_language'].str.contains('swe', na = False) |
         df['original_language'].isnull()) , 'original_language'] = 'other'

#organization
school_names = ['Centria', 'Metropolia', 'Laurea', 'Arcada', 'Häme', 'Karelia', 'Novia', 'Åland', 'Haaga-Helia',
                'Vaasa', 'Satakunta', 'Saimaa', 'Tampere', 'Turku', 'Jyväskylä', 'Kymenlaakso', 'Mikkeli', 'Savonia',
                'Kajaani', 'South-Eastern Finland', 'Lapland', 'LAB']

school_names_2 = {'Satakunnan' : 'Satakunta University of Applied Sciences',
                  'Lapin': 'Lapland University of Applied Sciences',
                  'Turun': 'Turku University of Applied Sciences',
                  'Diakonia': 'Diaconia University of Applied Sciences',
                  'Daikonia': 'Diaconia University of Applied Sciences',
                  'Lahden': 'Lahti University of Applied Sciences',
                  'Seinäjoen': 'Seinäjoki University of Applied Sciences',
                  'Keski-Pohjanmaan': 'Centria University of Applied Sciences',
                  'Humanistinen': 'Humak University of Applied Sciences',
                  'Pohjois-Karjalan': 'Karelia University of Applied Sciences',
                  'Rovaniemen': 'Rovaniemi University of Applied Sciences',
                  'Kaakkois-Suomen': 'South-Eastern Finland University of Applied Sciences',
                  'Poliisiammattikorkeakoulu': 'Police University College',
                  'Pirkanmaan' : 'Pirkanmaa University of Applied Sciences',
                  'Oulu' : 'Oulu University of Applied Sciences',
                  'tekstiili ja kemiantekniikan koulutusohjelma' : np.NaN,
                  'Degree programme' : np.NaN}

df.loc[df['organization'].isnull(), 'organization'] = np.NaN
for school_name in school_names:
    df.loc[df['organization'].str.contains(school_name, na = False, case = False), 'organization'] = school_name + ' University of Applied Sciences'
for school_name in school_names_2:
    df.loc[df['organization'].str.contains(school_name, na = False), 'organization'] = school_names_2[school_name]

In [4]:
#Keep samples with more than 135 words
df2 = df.loc[df['len'] > 135].copy()
df2.drop(columns = ['len'], axis = 1, inplace = True)

In [5]:
print('Final data length:', len(df2))
print('Deleted', len(df) - len(df2), 'samples')

Final data length: 174955
Deleted 1783 samples


In [6]:
#Save datafile pickle
filename = 'final_data'
outfile = open(filename, 'wb')
pickle.dump(df2, outfile, protocol = 3) #Old protocol
outfile.close()

In [7]:
#Test the saved file
infile = open(r'final_data','rb')
temp = pickle.load(infile)
infile.close()

In [8]:
temp.sample(5)

,handle,year,original_language,organization,google_translated_en,en,google_translated_fi,fi
130094,10024/157636,2018,fin,Tampere University of Applied Sciences,0,"In recent years, the use of wood has been incr...",0,Puun käyttö ja sen myötä puunkorjuu on lisäänt...
53118,10024/59693,2013,fin,Diaconia University of Applied Sciences,0,Atopic dermatitis sick children and their fami...,0,Ihoatopiaa sairastavan lapsen ja hänen perheen...
27568,10024/42064,2012,fin,Tampere University of Applied Sciences,0,The purpose of this thesis was to find out nur...,0,Tämän opinnäytetyön tarkoituksena oli selvittä...
75696,10024/94127,2015,eng,Metropolia University of Applied Sciences,0,Magento is a smart platform for E-commerce sol...,1,Magento on älykäs alusta verkkokaupan ratkaisu...
3293,10024/5148,2009,fin,Vaasa University of Applied Sciences,0,This thesis was made to Citec Information Oy A...,0,Tämä opinnäytetyö tehtiin Citec Information Oy...
